# Transposal script for the World Development Index dataset

As the original form of the WDI dataset is not suited for modelling (each feature is on a different row) and since it contains duplicated data, such as country names and codes, we must clean said data and reshape the dataset so all indicators for a certain country and year are in the same sample.

Mount GDrive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Import necessary libraries

In [0]:
import pandas as pd
import numpy as np

Compute transposed dataset header

In [0]:
indicators = pd.read_csv('/content/gdrive/My Drive/An 1 Masters/PML/Indicators.csv')

indicators_columns = list(indicators.columns)

unique_indicator_names = indicators.IndicatorName.unique()

processed_indicators_header = np.concatenate(([indicators_columns[0]], 
                                              unique_indicator_names,
                                              [indicators_columns[4]]))
  
indicators_dict = {key: [] for key in processed_indicators_header}    

Parse the initial dataset and append all features to a single list, with repsect to the country and year of the observations. This list will turn into a sample in the transposed dataset.

In [0]:
country = 'thisisnotacountry'
current_length = 0

for index, row in indicators.iterrows():
    
    if country != row[0]:
        indicators_dict[indicators_columns[0]].append(row[0])
        indicators_dict[indicators_columns[4]].append(row[4])
        
        if country != 'thisisnotacountry':
            for key, value in indicators_dict.items():
                if len(value) == current_length:
                    value.append(0)
            current_length += 1

        country = row[0]

    indicators_di ct[row[2]].append(row[5])
    
for key, value in indicators_dict.items():
    if len(value) == 13831:
        del value[-1]

Save the transposed data set as a pandas dataframe

In [0]:
transposed_indicators = pd.DataFrame(indicators_dict)

columns = transposed_indicators.columns[1:]

transposed_indicators[columns] = transposed_indicators[columns].apply(pd.to_numeric)

transposed_indicators.replace(0, np.nan, inplace=True)

Sort by country and year

In [0]:
transposed_indicators = transposed_indicators.sort_values(['CountryName', 'Year'], ascending=[True, True])

Save to drive

In [0]:
transposed_indicators.to_csv('/content/gdrive/My Drive/An 1 Masters/PML/transposed_indicators.csv', index = False)